# Задание по теме keras, simple ANN

ДЗ - обучить при помощи tf (keras) API любое ДЗ по обучению классических моделей.

Мы выбираем датасет Walmart.csv и с помощью keras решаем задачу регрессии, то есть по имеющимся данным построим модель, описывающую пятничные продажи в зависимости от набора параметров.

- План работы: <br>
Следует загрузить файл csv датасета.<br>
Произвести предобработку данных (выкинуть строки с пропусками). <br>
Выделить независимые переменные - характеристики (features) модели.<br>
Выделить колонку целевой (target) переменной - суммы пятничных продаж.<br>
Разделить данные на тренировочный и тестовый датасет.<br>
Создать модель нейронной сети с использованием keras.<br>
Обучить нейросеть.<br>
Провести проверку нейросети на тестовой выборке. Рассчитать метрики на тестовой выборке. <br>

Загрузка необходимых библиотек

In [12]:
%reset
# Загрузка необходимых библиотек
import pandas as pd

Nothing done.


Загрузка данных

In [13]:
# Загрузка данных
data = pd.read_csv('Walmart.csv')

# Размер таблицы данных
print(f' Table dimensions: {data.shape}')

# Просмотр первых 15 строк
data.head(7)

 Table dimensions: (6435, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
5,1,12-03-2010,1439541.59,0,57.79,2.667,211.380643,8.106
6,1,19-03-2010,1472515.79,0,54.58,2.720,211.215635,8.106


In [14]:
# Выкидываем пропуски на тот случай, если они есть. Реально их нету :)
# data.dropna()

Среди данных есть колонка с датами. Для построения модели регрессии дата не нужна.
Остальные параметры - номер магазина, совпадение с праздником, температура, стоимость топлива, индекс потребительских цен, уровень безработицы, - являются потенциально важными для модели.

In [15]:
# Удаление колонки с датами
data = data.drop('Date', axis = 1)

In [16]:
# Выделение целевой переменной
target = data['Weekly_Sales']  # Замените на реальное название колонки
# Выделение признаков
features = data.drop('Weekly_Sales', axis = 1) # Замените на реальные названия колонок


Разделение данных на тренировочный и тестовый наборы

In [17]:
from sklearn.model_selection import train_test_split

# Приведение данных к стандартному виду:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(features_std, target, test_size=0.2, random_state=42)

Создание модели нейронной сети с использованием Keras

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Создание модели
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Выходной слой для регрессии

c:\Users\varaksa_yua\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Компиляция модели

In [19]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

optimizer = Adam(learning_rate = 2.0)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

Добавляем коллбэк для уменьшения скорости обучения по мере обучения и второй коллбэк для остановки обучения при выходе на плато.
Второй реально не использовали - слишком рано, бывает, останавливает счет.

In [20]:
# Create the ReduceLROnPlateau callback
patience = 25
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = patience, min_lr=0.0001)
# Create the EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience = 3 * patience, restore_best_weights=True)

Обучение модели

In [21]:
# Обучение модели
batch_size = X_train.shape[0]
model.fit(X_train, y_train, epochs=1500, batch_size = batch_size, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step - loss: 1043939.0000 - mean_absolute_error: 1043939.0000 - val_loss: 1047045.1250 - val_mean_absolute_error: 1047045.1250 - learning_rate: 2.0000
Epoch 2/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1041702.6875 - mean_absolute_error: 1041702.6875 - val_loss: 1036202.8125 - val_mean_absolute_error: 1036202.8125 - learning_rate: 2.0000
Epoch 3/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1030556.3125 - mean_absolute_error: 1030556.3125 - val_loss: 1007826.1875 - val_mean_absolute_error: 1007826.1875 - learning_rate: 2.0000
Epoch 4/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1001430.5625 - mean_absolute_error: 1001430.5625 - val_loss: 951760.2500 - val_mean_absolute_error: 951760.2500 - learning_rate: 2.0000
Epoch 5/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 943931.0625 - mean_absolute_error: 943931.0625 - val_loss: 858217.1875 - val_mean_absolute_error: 858217.1875 - learning_rate: 2.0000
Epoch 6/1500
1/1 ━━━

Оценка модели на тестовой выборке

In [22]:
from sklearn.metrics import root_mean_squared_error 
from sklearn.metrics import r2_score

# Предсказание на тренировочной выборке
y_pred = model.predict(X_train)
# Вычисление среднеквадратичной ошибки
rmse = root_mean_squared_error(y_train, y_pred)
print(f'Root Mean Squared Error: {rmse}')
# Вычисление R^2
r2 = r2_score(y_train, y_pred)
print(f'R^2 Score: {r2*100:.2f}')


# Предсказание на тестовой выборке
y_pred = model.predict(X_test)
# Вычисление среднеквадратичной ошибки
rmse = root_mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {rmse}')
# Вычисление R^2
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2*100:.2f}')

161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step
Root Mean Squared Error: 398397.5857612943
R^2 Score: 50.01
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step
Root Mean Squared Error: 405540.6624781289
R^2 Score: 48.95


Выводы: <br>
На основе датасета Walmart, описывающего пятничные продажи в 45 магазинах соответствующей сети за период в несколько лет, и с помощью библиотеки keras составлена модель полносвязной нейронной сети, решающая задачу регрессии , позволяющая прогнозировать продажи для каждого из этих магазинов в зависимости от следующих параметров: 
- совпадение даты с праздничными днями
- температура воздуха
- стоимость топлива
- текущий индекс потребительских цен
- текущий уровень безработицы <br>

Размер батча мы устанавливали равным размеру всей тестовой выборки, потому как счет происходит довольно быстро. <br>
Использовалось адаптивное уменьшение скорости обучения, начиная с learning rate = 2: <br>
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001) <br>
Количество эпох бралось равным 1500. <br>

Отметим, что выбор оптимальной конфигурации сети осложняется случайным характером результатов обучения: <br>
так, для очень простых сетей с одним или несколькими нейронами в двух скрытых слоях могла наблюдаться ситуация, когда при последовательных запусках сеть демонстрировала то полный провал (R^2 < 0), то какую-то минимальную производительность (R^2 ~ 10-12%). <br>
Для более сложных сетей не раз наблюдалась ситуация, когда при одних и тех же параметрах мы могли иногда получить R^2 на уровне 70-80%, а затем около 17%.

Далее показано, как менялись наши результаты по мере усложнения сети:

Для сети 1-1-1 положительный R^2 не достигается, даже при увеличении начальной скорости обучения до 1000 и количества итераций до 3000 <br>
Конфигурация неработоспособна. <br>

Для сети 2-2-1: <br>
Root Mean Squared Error: 535221.490937713 <br>
R^2 Score: 11.08 <br>
Однако часто R^2 отрицательный. Делаем вывод, что конфигурация в целом неработоспособна. <br>

Для сети 2-4-1: <br>
Root Mean Squared Error: 535106.879335905 <br>
R^2 Score: 11.12 <br>

Для сети 4-8-1: <br>
Root Mean Squared Error: 526060.5365927957
R^2 Score: 14.10

Для сети 8-16-1: <br>
Root Mean Squared Error: 516257.9102990206
R^2 Score: 17.27

Для сети 16-32-1: <br>
Root Mean Squared Error: 340196.39360479533
R^2 Score: 64.08

Раннее, с другими параметрами, мы доходили до следующих значений, но повторить их не удалось:
Для сети 16-32-1. <br>
Root Mean Squared Error: 177020.51276464973 <br>
R^2 Score: 90.27 <br>

Для сети 32-64-1 достигли: <br>
Root Mean Squared Error: 220658.97665095868 <br>
R^2 Score: 84.89 <br>

Для простой сети 8-16-1 достигли <br>
Root Mean Squared Error: 307517.8225739074 <br>
R^2 Score: 70.65 <br>

Отметим, что в свое время, в домашней работе 17, в случае подбора параметров с помощью optuna и работы в библиотеках pandas + sklearn мы получили объясненную дисперсию на уровне 0.999 и среднеквадратичную ошибку предсказания на тестовой выборке порядка 8 тыс. денежных единиц. 